In [1]:
%matplotlib inline

import os
import os.path as osp
import sys
# import Image
# import sys
import argparse
import os, tempfile, glob, shutil
import random
from datetime import datetime
random.seed(datetime.now())

# import gaussian_process
import GPy
import GPyOpt
import subprocess as sub
import numpy as np
import scipy.spatial
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy import optimize
import time

global g_blender_executable_path,  blank_file, render_code, model_file, table_file, temp_dirname, epsilon, \
       filename, time, location_gt, rotation_gt


epsilon = 1e-6


if os.environ.get('BLENDER_PATH') == None:
    print("Please set BLENDER_PATH in bashrc!")
    sys.exit()

g_blender_executable_path = os.environ['BLENDER_PATH']

obj_model_folder = 'obj_models/'
table_model_folder = 'table_models/'

g_blank_blend_file_path = 'blank.blend'
syn_images_folder = 'rendered_images'
if not os.path.exists(syn_images_folder):
    os.mkdir(syn_images_folder)

all_models = []
for file in os.listdir(obj_model_folder):
    if file.endswith(".obj"):
        all_models.append(file)

print(all_models)

# random select model in all_models
# To Do

# model_name and table_name are hard coded
model_name = 'expo_dry_erase_board_eraser'
robot_name = 'reflex'

########################### Input and GT ####################
fps = 24

path = '/home/zhusj/Dropbox/16fall/learningPhysics/data/data/trial4/'

filelist = ['exp1', 'exp2', 'exp3']
timelist = [8,4,1]

location_gt = []
rotation_gt = []

for file in filelist:
    count = 0
    filename = path+file+'.txt'
    with open(filename) as f:

        obj_start_config = []
        hand_start_config = []
        obj_end_config = []
        hand_end_config = []
        for line in f: # read rest of lines
            if count == 7: 
                obj_end_config.append([float(x) for x in line.split()])
            count += 1

    print obj_end_config
    myorder=[0,1,2,6,3,4,5]
    obj_end_config = [ obj_end_config[0][i] for i in myorder]
    print obj_end_config


    obj_end_config = np.array(obj_end_config)
    print obj_end_config.shape

    location_gt.append(obj_end_config[:3])
    rotation_gt.append(obj_end_config[3:])

print location_gt, rotation_gt
##############################################

table_name = 'table'
arm_name ='arm'
image_name = 'demo_img.png'
output_img = osp.join(syn_images_folder, image_name)
output_blend = osp.join(syn_images_folder, image_name+'.blend')

# call blender to render images
blank_file = osp.join(g_blank_blend_file_path)
render_code = osp.join('render_physics_reflex_expo.py')
model_file = osp.join(obj_model_folder, model_name+ '.obj')
robot_file = osp.join(obj_model_folder, robot_name+ '.obj')
table_file = osp.join(table_model_folder, table_name+ '.obj')
# print render_code
# print model_file
temp_dirname = '/home/zhusj/repos/blender_scene_generator/test'  #tempfile.mkdtemp()

/home/zhusj/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


['dasani_water_bottle.obj', 'expo_dry_erase_board_eraser.obj', 'reflex.obj', 'pliers.obj', 'folgers_classic_roast_coffee.obj', 'expo_dry_erase_board_eraser_00000.obj', 'reflex_nc.obj', 'crayola_24_ct.obj', 'robotiq.obj', 'scotch_duct_tape.obj', 'folgers_classic_roast_coffee_00000.obj', 'kleenex_tissue_box.obj']
[[0.543561, -0.123172, 0.54746, -0.00114687, 0.00659373, -0.781483, 0.62389]]
[0.543561, -0.123172, 0.54746, 0.62389, -0.00114687, 0.00659373, -0.781483]
(7,)
[[0.532153, -0.0257914, 0.548445, -0.00623893, -0.00737948, 0.902664, -0.430238]]
[0.532153, -0.0257914, 0.548445, -0.430238, -0.00623893, -0.00737948, 0.902664]
(7,)
[[0.48143, -0.0221645, 0.54965, -0.00235522, -0.00839558, 0.972091, -0.234441]]
[0.48143, -0.0221645, 0.54965, -0.234441, -0.00235522, -0.00839558, 0.972091]
(7,)
[array([ 0.543561, -0.123172,  0.54746 ]), array([ 0.532153 , -0.0257914,  0.548445 ]), array([ 0.48143  , -0.0221645,  0.54965  ])] [array([ 0.62389   , -0.00114687,  0.00659373, -0.781483  ]), arr

In [2]:
def run_physics_engine(filename, mass, friction, force, time):

    location =[]
    rotation = []    

    render_cmd = '%s %s -b --python %s -- %s %s %s %s %s %s %s %s %s' % \
    (g_blender_executable_path, blank_file, render_code, robot_file, model_file, time, \
    filename, force_gt, mass, friction, table_file, temp_dirname)

    p = os.popen(render_cmd,"r")
    while 1:
        line = p.readline()
        if not line: break
        if "location" in line: 
            print "output:", line
            vals = line.split(',')
            count = 0
            for val in vals:
                count += 1
                if count == 1:
                    x= float(val[-6:])
                if count == 2:
                    y= float(val[-7:])

        if "rotation" in line: 
            print "output:", line
            vals = line.split(',')
            count = 0
            for val in vals:
                count += 1
                if count == 1:
                    qw= float(val[-6:])
                if count == 2:
                    qx= float(val[-6:])
                if count == 3:
                    qy= float(val[-6:])
                if count == 4:
                    newstr = val[-10:-3].replace("=", "")
                    qz= float(newstr)
                    
    location.append([x, y])
    location = np.asarray(location)
    rotation.append([qw,qx,qy,qz])
    rotation = np.asarray(rotation)
    return location, rotation

In [3]:
def myf(X):

    mass = X[0][0]
    friction = X[0][1]
    
    y_new = 0
    count = 0
    for file in filelist:        
        filename = path+file+'.txt'
        time = timelist[count]*fps
#         print 'input: ', filename, mass, friction, force_gt, time
        
        location, rotation = run_physics_engine(filename, mass, friction, force_gt, time)  
        dist = np.sqrt(np.sum(((location-location_gt[count][0:2])**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt[count]))**2
        

        print dist, q_dist
        y_new += dist + q_dist
        count += 1

    print "X, y_new",X,y_new

    return y_new

In [7]:
def myf_GS(X):

    mass = X[0]
    friction = X[1]

    y_new = 0
    count = 0
    for file in filelist:        
        filename = path+file+'.txt'
        time = timelist[count]*fps
#         print 'input: ', filename, mass, friction, force_gt, time
        
        location, rotation = run_physics_engine(filename, mass, friction, force_gt, time)  
        dist = np.sqrt(np.sum(((location-location_gt[count][0:2])**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt[count]))**2
        

        print dist, q_dist
        y_new += dist + q_dist
        count += 1
#     y_new = np.max([dist1, dist2, dist3]) 
    print "X, y_new",X,y_new


    return y_new

In [11]:
BO_result = []
GS_result =[]
BO_time =[]
GS_time =[]
mass_GT =[]
friction_GT =[]

force_gt = float(48)

for count in xrange(1,11):
    random.seed(datetime.now())
    ################## Grid Search #############
    rranges = (slice(0.0, 0.5, 0.05), slice(0.0, 0.7, 0.05))


    start_time = time.time()

    x0,fval,grid,jout = optimize.brute(myf_GS, rranges, full_output=True, finish=None)

    elapsed_time = time.time() - start_time
#     print elapsed_time

    GS_result.append([x0,fval,grid,jout])
    GS_time.append(elapsed_time)
    
#     print myBopt.x_opt, myBopt.fx_opt
#     print myBopt.cum_time
#     print x0, fval    
#     print elapsed_time

# exp1: 0.543561 -0.123172 0.54746 0.62389 -0.00114687 0.00659373 -0.781483
# exp2: 0.532153 -0.0257914 0.548445 -0.430238 -0.00623893 -0.00737948 0.902664
# exp3: 0.48143 -0.0221645 0.54965 -0.234441 -0.00235522 -0.00839558 0.972091 

# exp8: 0.423403 0.229149 0.558357 -0.232128 -0.00738495 -0.0275467 0.972267 

output: location:  <Vector (0.5540, -0.2101, 0.7750)>

output: rotation:  <Quaternion (w=0.7600, x=-0.0001, y=-0.0015, z=-0.6499)>

0.0875525551026 0.0355739132762
output: location:  <Vector (0.5436, -0.1230, 0.7750)>

output: rotation:  <Quaternion (w=0.6240, x=-0.0005, y=0.0068, z=-0.7814)>

0.0978802621725 0.0515899582988
output: location:  <Vector (0.5322, -0.0260, 0.7750)>

output: rotation:  <Quaternion (w=0.4302, x=0.0073, y=0.0073, z=-0.9027)>

0.0509146733295 0.0426521435268
X, y_new [ 0.  0.] 0.366163505706
output: location:  <Vector (0.5540, -0.2101, 0.7750)>

output: rotation:  <Quaternion (w=0.7600, x=-0.0001, y=-0.0015, z=-0.6499)>

0.0875525551026 0.0355739132762
output: location:  <Vector (0.5436, -0.1230, 0.7750)>

output: rotation:  <Quaternion (w=0.6240, x=-0.0005, y=0.0068, z=-0.7814)>

0.0978802621725 0.0515899582988
output: location:  <Vector (0.5322, -0.0260, 0.7750)>

output: rotation:  <Quaternion (w=0.4302, x=0.0073, y=0.0073, z=-0.9027)>

0.0509146733295 0.04

In [18]:
GS_result[0][3]

array([[ 0.36616351,  0.36616351,  0.36616351,  0.36616351,  0.36616351,
         0.36616351,  0.36616351,  0.36616351,  0.36616351,  0.36616351,
         0.36616351,  0.36616351,  0.36616351,  0.36616351],
       [ 1.96491334,  0.76095399,  0.41950601,  0.13179645,  0.11154412,
         0.09675562,  0.11246462,  0.126782  ,  0.12835335,  0.14667854,
         0.1378442 ,  0.09749741,  0.08474693,  0.10272412],
       [ 1.96491334,  0.76095399,  0.41950601,  0.13179645,  0.11154412,
         0.09675562,  0.11246462,  0.126782  ,  0.12835335,  0.14667854,
         0.1378442 ,  0.09749741,  0.08474693,  0.10272412],
       [ 1.9094155 ,  0.73274125,  0.43306346,  0.10591756,  0.11514634,
         0.11259345,  0.12785574,  0.11115349,  0.14311437,  0.14979556,
         0.16165194,  0.11997654,  0.11968241,  0.1470685 ],
       [ 1.96491334,  0.76095399,  0.41950601,  0.13179645,  0.11154412,
         0.09675562,  0.11246462,  0.126782  ,  0.12835335,  0.14667854,
         0.1378442 ,  0.09

In [19]:
GS_result

[[array([ 0.05,  0.6 ]),
  0.084746927459831145,
  array([[[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
            0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
          [ 0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
            0.05,  0.05,  0.05,  0.05,  0.05],
          [ 0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ,
            0.1 ,  0.1 ,  0.1 ,  0.1 ,  0.1 ],
          [ 0.15,  0.15,  0.15,  0.15,  0.15,  0.15,  0.15,  0.15,  0.15,
            0.15,  0.15,  0.15,  0.15,  0.15],
          [ 0.2 ,  0.2 ,  0.2 ,  0.2 ,  0.2 ,  0.2 ,  0.2 ,  0.2 ,  0.2 ,
            0.2 ,  0.2 ,  0.2 ,  0.2 ,  0.2 ],
          [ 0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25,
            0.25,  0.25,  0.25,  0.25,  0.25],
          [ 0.3 ,  0.3 ,  0.3 ,  0.3 ,  0.3 ,  0.3 ,  0.3 ,  0.3 ,  0.3 ,
            0.3 ,  0.3 ,  0.3 ,  0.3 ,  0.3 ],
          [ 0.35,  0.35,  0.35,  0.35,  0.35,  0.35,  0.35,  0.35,  0.35,
            0.35,  0.35,  0.35

In [13]:
GS_time

[5971.37624001503]